# Data wrangling

### What we'll cover

* Loading data
* Transforming it
* Storing it

### How we'll do it

We'll be working with a dataset, `contracts_data.csv`. It contains selected data on federal contracts from FY 2015 in Colorado. We'll use it to answer some real-life questions, with examples before each step. The goal is to have you write code you can use as a reference later on, so we'll try to explain what's happening.

## Loading data

### Loading from CSVs

The `agate` library (formerly `csvkit` and `journalism`), by ex-Tribune developer Chris Groskopf, was built with journalists handling CSVs in mind ([here's the documentation](https://agate.readthedocs.org/)). The fundamental unit in `agate` is the table, and there's a one-step method of creating a table from a CSV file:

    data = agate.Table.from_csv('data.csv')

#### Your turn

Load the data located in the `data/contracts_data.csv` file into an `agate` table and check out how the example works. Think about how you'd access a row's data. How would you answer a question like "sum up all the values of a column in this spreadsheet?"

In [3]:
import agate

table = # Your loading code goes here

print table

for row in table.rows[:10]: # This is a big file, so let's just show the first 10 rows
    for column in row.keys():
        print '%s: %s' % (column, row[column])
    print '<----------->'

SyntaxError: invalid syntax (<ipython-input-3-267fc909aa4b>, line 3)

It's worth mentioning that there's another way to load data from a CSV, and it may be more general-purpose: `csv.DictReader`. When you load a CSV file with `DictReader`, you'll get a list of dictionaries, one per line, with the header row (assuming there is one) converted into the dictionary's keys.

In [11]:
from csv import DictReader

csv_data = []

with open('data/contracts_data.csv') as fh:
    reader = DictReader(fh)
    for row in reader:
        # row is now a dict; each column in the CSV is a key in the dict
        csv_data.append(row)

print 'Found %d lines' % len(csv_data)

Found 2000 lines


### Loading from an API

### Loading from JSON

The `json` module includes two main methods: `loads` to load JSON data, and `dumps` to create JSON from Python objects. If you haven't worked with JSON before, it's a very convenient way of passing around data objects using pure text. 

`loads` just takes a single string of JSON-formatted data as an argument, and returns a Python object.

    data = json.loads('{"foo":"bar"}')
    print data['foo']

displays `bar`.

#### Your turn

Load the data located in the `data/contracts_data.json` file into a data object.

In [ ]:
import json

## Transforming data

### Summing up a column of data

### Filtering rows of data

### Sorting rows of data

### String cleaning

### Geocoding addresses

We'll be using the excellent library geopy ([docs](https://geopy.readthedocs.org/en/1.10.0/); run `pip install geopy` if you don't have it on your machine), which provides a common, simple interface to a variety of different geocoding services. It's worth noting that not all geocoding services work equally well, and they often have limits on how many requests you can make in a short amount of time. So if you're going to geocode a large number of addresses, you'll need to figure out which service is best for you.

To create an instance of a geocoder using a particular service, first import the appropriate class:

    from geopy.geocoders import Nominatim

Then create the instance:

    geocoder = Nominatim()

Once we have the geocoder instance created, using it is as simple as passing a string containing the address we're interested in:

    location = geocoder.geocode("1701 California Street, Denver, CO")

And from there:

    print location.latitude, location.longitude
    
Returns `39.7472023 -104.9904179`

### Your turn

Create an instance of the Google geocoder, and use it to find the latitude and longitude of Molly Brown's house at 1340 Pennsylvania St in Denver **TODO: Should we make this more relevant to the story data?**. (Heads up: most geocoding services restrict heavy usage via IP addresses, so this classroom might get temporarily blocked and the examples may not work).

In [ ]:
from geopy.geocoders import

### Comparing dates and date strings 

The standard `datetime` module and the excellent [strftime.org cheat sheet](http://www.strftime.org) (seriously, bookmark it) make Python able to translate between a really delicious variety of date and time formats.

To work with dates in our data, we first need to convert strings containing dates to actual date objects. To see why, let's ask the question: which of these dates comes first?

In [ ]:
older = '5-13-1989'
newer = '2010-06-17'

if older < newer:
    print "That's what I expect."
else:
    print "Huh?"

This is because, when Python is comparing strings to each other (and both of the above dates, despite looking date-like, are just strings of text) it defaults to comparing them alphabetically. Does the first letter of string A come before the first letter of string B? If so, A < B.

So, we need to tell Python how to convert our string of arbitrary text into a `datetime` object. Once we do that, we get all kinds of superpowers - we can add and subtract time from a date, compare dates to each other, adjust the timezone of our date, pull out just the month or year, determine what day of the week it was, and on and on.

The `datetime` module provides several types of date-related classes we can use (in particular, `date`, `time` and `datetime`, which combines the first two) but for now we'll just rely on `datetime`. Annoyingly, `datetime` is both the name of a module, and the name of a class within that module, so we have to do dumb stuff like this:

    from datetime import datetime

Or

    import datetime
    datetime.datetime.now()

I like the first one, myself. If we just wanted, say, `date` then we'd do:

    from datetime import date

Or

    import datetime
    datetime.date.today()

Then we need to determine how to understand the date objects we're working with in our data (and this is where [strftime.org](http://www.strftime.org) is really useful). We do this by creating a format string, which tells datetime how our dates are structured.

Take `older`, above. It's date is "5-13-1989": "month hyphen day hyphen 4-digit year". In the format string language that `datetime` uses, that translates to "%m (month) hyphen %d (day) hyphen %Y (4-digit year)". `datetime` expects that the format string will also tell it about any non-date characters, so we also have to include the hyphens in our format string. The end result will look like this:

    format_string = '%m-%d-%Y'

We then use the `strptime` function to create a `datetime` object from a string. We have to pass it both the string we'd like to convert, and the format string that tells us how to do so:

    dt = datetime.strptime('5-13-1989', format_string)

### Your turn

Convert the dates below into `datetime` objects. For bonus points, try converting them into `date` objects. What did you have to do differently? When might one be preferred over the other?

In [ ]:
from datetime import datetime

newer = '2010-06-17'
two_digit_year = '1/20/00' # Now you see why Y2K seemed like a big deal (does anyone even remember Y2K?)
crazy_text_having_variable = '2013 in June on day 12'

## Storing data

### Saving data as a CSV

### Saving data as JSON

The `json` module makes it as easy to write JSON as it is to read it. However, since JSON is a very verbose format, it can make for very large files. The `contracts_data.csv` file, above, is 1.4 MB (enough to fit on a floppy!) - but the exact same data, stored in JSON, is over three times larger - 4.6 MB.

So it's worth considering for a moment whether storing data in JSON is what you really want to do. It's great for Javascript web apps, for instance, because it's highly flexible and self-documenting, and therefore easy for programs to read it and work with it. But, particularly if your data has a large number of columns, the size of your JSON files can get very large very quickly (because every single row will repeat the name of every single column, plus 4 " characters and a : character).

Another thing to keep in mind is that some datatypes can't be represented in JSON. For instance, `datetime` objects need to be converted to strings first; you'll get an error if you try to save a `datetime` in JSON directly.

The `dumps` method of the `json` module is the exact opposite of the `loads` method we used above, to load JSON data. Think of `dumps` as meaning, "DUMP to String" and `loads` as meaning, "LOAD from String". Here, we just want to write one gigantic string to a file, 

In [10]:
import json

with open('data/contracts_data.json', 'w+') as fh:
    fh.write(json.dumps(csv_data))

### Saving data to S3